# IE 306  SYSTEM SIMULATION
## FURKAN KESKIN - 2018400150
## SINEM KOCOGLU - 2020400339

In [83]:
import numpy as np
from queue import Queue, PriorityQueue

SEED = 2018400150 + 2020400339

nurses = 4  # S
beds = 6    # K
p_1 = 0.25
lambd = 1
mu_T = 0.3125
mu_s = 0.16
mu_cb = 0.1666666667

In [65]:
def Generate_Interarrival():
    return np.random.exponential(1/lambd, 50)

def Generate_Nurse_Service_Time():
    return np.random.exponential(1/mu_T, 50)

def Generate_Hospital_Healing_Time():
    return np.random.exponential(1/mu_cb, 50)

def Generate_Home_Healing_Time(condition):
    if condition == 's':
        return np.random.exponential(1/mu_s, 50)
    elif condition == 'c':
        alfa = np.random.uniform(1.25, 1.75)
        return np.random.exponential(alfa/mu_cb, 50)
    

def Arrival():
    

In [84]:
np.random.seed(SEED)

interarrivals = Generate_Interarrival()
nurse_service_times = Generate_Nurse_Service_Time()
hospital_healing_times = Generate_Hospital_Healing_Time()
home_healing_times_stable = Generate_Home_Healing_Time('s')
home_healing_times_critical = Generate_Home_Healing_Time('c')
arrivals = interarrivals.cumsum()

print(interarrivals)
print(arrivals)

[2.41070487e-01 6.47407769e-01 4.50411894e+00 1.07434257e+00
 2.53375056e+00 3.19182038e-01 1.10184572e+00 5.85289474e-01
 2.28080274e-01 8.26462025e-02 7.52756332e-01 3.82814961e-01
 8.08808754e-01 6.04292502e-02 1.84175342e+00 1.53742158e+00
 3.46427665e-01 1.53435066e-02 2.19875851e-01 4.21094336e+00
 1.47260307e+00 3.26912281e-01 7.63602801e-02 6.63089981e-01
 8.45308865e-02 7.50748687e-01 9.69871693e-02 1.84875141e-01
 2.58576118e+00 8.51753397e-01 6.79433616e-01 7.58412151e-01
 2.45503843e-03 3.42977880e+00 1.72511183e+00 2.11595757e-01
 5.41296407e-01 1.41057061e+00 6.48464237e-01 1.34652378e+00
 2.32358741e-01 7.08265163e-01 1.40769545e+00 3.70437850e+00
 1.54393276e-01 3.57141439e-01 7.35832352e-02 4.76161819e-01
 1.48732732e+00 5.73864356e-02]
[ 0.24107049  0.88847826  5.3925972   6.46693977  9.00069033  9.31987237
 10.42171809 11.00700757 11.23508784 11.31773404 12.07049038 12.45330534
 13.26211409 13.32254334 15.16429676 16.70171834 17.04814601 17.06348951
 17.28336536 21.4

In [ ]:
FEL = PriorityQueue() # Future Event List  
WQ = Queue()          # Waiting Queue for Nurse Service
busy_nurses = 0       # System starts with a patient with ID = 1 in simulation time = 0
occupied_beds = 0
last_departed_id = 0
simulation_time = 0

def Arrival(pid):
    if pid == 0:
        busy_nurses += 1
        FEL.push(arrivals[pid])
        FEL.push(simulation_time + nurse_service_times[pid])
    else:     
        # there is an available nurse
        if busy_nurses < nurses:
            

            busy_nurses += 1
    